In [4]:
# from llama_index.indices.postprocessor import SentenceTransformerRerank
# rerank = SentenceTransformer Rerank(top_n=2, model="BAAI/bge-reranker-base")

In [5]:
from scripts import utils
import os
import openai

openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [6]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["pdfs/eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [7]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'>
Doc ID: cf887f7e-3cc0-4eab-ba9f-8fb5de9f5cbf
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


In [8]:
# documents[1].text

### Basic RAG Pipeline

In [9]:
from pprint import pprint

In [10]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [11]:
# pprint(document.text)

In [12]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

In [13]:
query_engine = index.as_query_engine()

In [14]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
pprint(str(response))

('When finding projects to build your experience, there are several steps you '
 'can take. First, you can join existing projects by asking to join someone '
 "else's project if they have an idea. Additionally, you can continue reading, "
 'taking courses, and talking to domain experts to come up with new ideas. It '
 'is also helpful to focus on a specific application area where machine '
 'learning has not yet been applied. This can give you the opportunity to '
 'explore unique and creative applications that no one else has done yet. '
 'Finally, you can develop a side hustle or personal project that may not '
 'initially be part of your job but can stir your creative juices and '
 'strengthen bonds with collaborators.')


### Eval setup using TruLens

In [15]:
eval_questions = []
with open('text/eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character 
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?


In [16]:
# You can try your own question:
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)

In [17]:
pprint(eval_questions)

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter '
 'syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?',
 'What is the right AI job for me?']


In [18]:
from trulens_eval import Tru
tru = Tru(database_url="sqlite:///db/default.sqlite")

# tru.reset_database()

Tru was already initialized. Cannot change database_url=sqlite:///db/default.sqlite or database_file=None .


In [19]:
from scripts.utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

In [20]:
import time

with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

        # time.sleep(10)

In [21]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [22]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_e9af2ee8eee02f09889a0dc01be290a4,"""What are the keys to building a career in AI?""","""The keys to building a career in AI are learn...",-,"{""record_id"": ""record_hash_e9af2ee8eee02f09889...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-29T10:44:28.042939"", ""...",2024-01-29T10:44:46.837454,1.0,0.6,1.000,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'prompt': 'What are the keys to bui...,"[{'args': {'source': 'PAGE 1Founder, DeepLearn...",18,2188,0.003367
1,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_21509cda6b8f6270016657c151f3f062,"""How can teamwork contribute to success in AI?""","""Collaborating and working in teams is crucial...",-,"{""record_id"": ""record_hash_21509cda6b8f6270016...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-29T10:44:47.734365"", ""...",2024-01-29T10:44:59.676200,0.9,0.0,0.900,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'Hopefully the previous c...,11,1713,0.002613
2,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_3861c8261f5ea70fa16680fd75f6e082,"""What is the importance of networking in AI?""","""Networking is important in AI because it allo...",-,"{""record_id"": ""record_hash_3861c8261f5ea70fa16...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-29T10:45:00.651377"", ""...",2024-01-29T10:45:12.176058,0.9,0.1,0.250,[{'args': {'prompt': 'What is the importance o...,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': 'Hopefully the previous c...,11,1706,0.002599
3,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_3e6634f11cea37e9107f26a1e216c8aa,"""What are some good habits to develop for a su...","""Developing good habits is crucial for a succe...",-,"{""record_id"": ""record_hash_3e6634f11cea37e9107...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-29T10:45:13.143374"", ""...",2024-01-29T10:45:25.703314,1.0,0.6,0.875,[{'args': {'prompt': 'What are some good habit...,[{'args': {'prompt': 'What are some good habit...,[{'args': {'source': 'Hopefully the previous c...,12,1676,0.002554
4,Direct Query Engine,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_b798a1ed38d256b5ccf0a48888e811ec,"""How can altruism be beneficial in building a ...","""Altruism can be beneficial in building a care...",-,"{""record_id"": ""record_hash_b798a1ed38d256b5ccf...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-01-29T10:45:26.638393"", ""...",2024-01-29T10:45:38.322263,0.9,0.3,0.500,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'prompt': 'How can altruism be bene...,[{'args': {'source': 'Hopefully the previous c...,11,1668,0.002539


In [23]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.1.74:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### Advanced RAG Pipeline

In [24]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [25]:
from scripts.utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [26]:
from scripts.utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

c:\Users\ve797\anaconda4\envs\openai\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ve797\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [27]:
window_response = sentence_window_engine.query(
    "how do I get started on a personal project in AI?"
)
pprint(str(window_response))

('To get started on a personal project in AI, it is important to first '
 'identify and scope the project. Consider your career goals and choose a '
 'project that complements them. Ensure that the project is responsible, '
 'ethical, and beneficial to people. As you progress in your career, aim for '
 'projects that grow in scope, complexity, and impact over time. Building a '
 'portfolio of projects that shows skill progression can also be helpful. '
 'Additionally, there are resources available in the provided chapters that '
 'can guide you in starting your AI project and finding the right job in the '
 'field.')


In [28]:
# tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [29]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

What are the keys to building a career in AI?
The keys to building a career in AI are learning foundational technical skills, working on projects, and finding a job, all of which is supported by being part of a community.
How can teamwork contribute to success in AI?
Teamwork can contribute to success in AI by allowing individuals to leverage the expertise and insights of their colleagues. When working on larger AI projects that require collaboration, the ability to lead and work effectively as a team becomes crucial. By working together, team members can share their deep technical insights, make informed decisions about technical architecture or data collection, and ultimately improve the project. Additionally, being surrounded by colleagues who are dedicated, hardworking, and continuously learning can inspire individuals to do the same, leading to greater success in AI endeavors.
What is the importance of networking in AI?
Networking is important in AI because it allows individuals t

In [30]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
Sentence Window Query Engine,0.8875,0.870000,0.385000,14.272727,0.000846
Direct Query Engine,0.7250,0.927273,0.263636,14.272727,0.003064


In [31]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.1.74:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### Auto-merging retrieval

In [32]:
from scripts.utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

In [33]:
from scripts.utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [34]:
auto_merging_response = automerging_query_engine.query(
    "How do I build a portfolio of AI projects?"
)
print(str(auto_merging_response))

To build a portfolio of AI projects, it is important to start with simple undertakings and gradually progress to more complex ones over time. This progression will demonstrate your growth and development in the field. Additionally, effective communication is key in building a portfolio. You should be able to explain your thinking and the value of your work to others. This will help others see the potential in your projects and gain their trust. By effectively communicating your ideas, you can also receive feedback and identify any flaws in your reasoning.


In [35]:
# tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [36]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x14dc7d28910 is calling an instrumented method <function BaseQueryEngine.query at 0x0000014DB229BA60>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x14dcd9f79d0) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x14dc7d28910 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x0000014DB8646FC0>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x14dcd9f79d0) using this function.
A new object of type <class 'llama_index.retrievers.auto_merging_retriever.AutoMergingRetriever'> at 0x14dc7d1a290 is calling an instrumented method <function BaseRetriever.retrieve at 0x0000014DB4CFFF60>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on other obje

What are the keys to building a career in AI?
The keys to building a career in AI are learning foundational technical skills, working on projects, finding a job, and being part of a community.


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x14dc7d28910 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x0000014DB8646FC0>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x14dcd9f79d0) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x14dc7d2b910 is calling an instrumented method <function Refine.get_response at 0x0000014DB4CFDD00>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x14dcd9f78d0) using this function.


How can teamwork contribute to success in AI?
Teamwork can contribute to success in AI by allowing individuals to work on larger projects that require collaboration and coordination. When working on AI projects that involve teamwork, the ability to lead and work effectively with others becomes crucial. By leveraging the diverse skills and expertise of team members, AI projects can benefit from different perspectives and ideas, leading to more innovative solutions. Additionally, teamwork enables individuals to divide tasks and responsibilities, increasing efficiency and productivity. Overall, teamwork in AI projects can lead to better outcomes, improved problem-solving, and the ability to tackle more significant and complex projects.


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x14dc7d28910 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x0000014DB8646FC0>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x14dcd9f79d0) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x14dc7d2b910 is calling an instrumented method <function Refine.get_response at 0x0000014DB4CFDD00>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x14dcd9f78d0) using this function.


...

> Merging 3 nodes into parent node.
> Parent node id: effdd48f-8644-432d-96a6-594b12f33540.
> Parent node text: PAGE 37Overcoming Imposter 
SyndromeCHAPTER 11

PAGE 38Before we dive into the final chapter of t...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x14dc7d2b910 is calling an instrumented method <function Refine.get_response at 0x0000014DB4CFDD00>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x14dcd9f78d0) using this function.


Is it normal to find parts of AI challenging?
Yes, it is normal to find parts of AI challenging. The author mentions that even accomplished individuals in the AI community have struggled with technical challenges at some point. They share their own experiences of struggling to understand certain concepts and making mistakes in their implementation. Therefore, it is clear that finding parts of AI challenging is a common experience.


A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x14dc7d28910 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x0000014DB8646FC0>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x14dcd9f79d0) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x14dc7d2b910 is calling an instrumented method <function Refine.get_response at 0x0000014DB4CFDD00>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x14dcd9f78d0) using this function.


What is the right AI job for me?
The right AI job for you would depend on your specific career goals and background. It could be a role that aligns with your current skills and experience, or it could be a role that allows you to transition from a different field into AI. It is important to consider your interests, strengths, and the skills you want to develop in order to find the right AI job for you.


In [37]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
Sentence Window Query Engine,0.897727,0.863636,0.400000,14.272727,0.000846
Automerging Query Engine,0.866667,0.910000,0.410000,14.272727,0.000945
Direct Query Engine,0.725000,0.927273,0.263636,14.272727,0.003064


In [40]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.1.74:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [41]:
# tru.stop_dashboard()